# Preliminary Data Exploration Justice40 

In [14]:
# Load packages
import pandas as pd
import geopandas as gpd
import numpy as np

# Remove max columns
pd.set_option('display.max_columns', None)


In [25]:
# Import data
# usa = pd.read_csv("data/usa.csv")
# v1 = gpd.read_file("data/1.0-shapefile-codebook/usa/usa.shp")
# v2 = gpd.read_file("data/2.0-shapefile-codebook/usa/usa.shp")


In [15]:
usa.head()

,GEOID10_TRACT,Does the tract have at least 35 acres in it?,Contains agricultural value,Names of Tribal areas within Census tract,Housing burden (percent),Share of homes with no kitchen or indoor plumbing (percent),Total population,Median household income (% of state median household income),Current asthma among adults aged greater than or equal to 18 years,Coronary heart disease among adults aged greater than or equal to 18 years,Cancer (excluding skin cancer) among adults aged greater than or equal to 18 years,Current lack of health insurance among adults aged 18-64 years,Diagnosed diabetes among adults aged greater than or equal to 18 years,Physical health not good for greater than or equal to 14 days among adults aged greater than or equal to 18 years,Percent of individuals < 100% Federal Poverty Line,Percent of individuals < 150% Federal Poverty Line,Percent of individuals below 200% Federal Poverty Line,Area Median Income (State or metropolitan),Median household income in the past 12 months,Energy burden,FEMA Risk Index Expected Annual Loss Score,Urban Heuristic Flag,Air toxics cancer risk,Respiratory hazard index,Diesel particulate matter exposure,PM2.5 in the air,Ozone,Traffic proximity and volume,Proximity to Risk Management Plan (RMP) facilities,Proximity to hazardous waste sites,Proximity to NPL sites,Wastewater discharge,Percent pre-1960s housing (lead paint indicator),Individuals under 5 years old,Individuals over 64 years old,Linguistic isolation (percent),Percent of households in linguistic isolation,Poverty (Less than 200% of federal poverty line),Percent individuals age 25 or over with less than high school degree,Unemployment (percent),Median value ($) of owner-occupied housing units,Percent enrollment in college or graduate school,Percent of population not currently enrolled in college or graduate school,Expected building loss rate (Natural Hazards Risk Index),Expected agricultural loss rate (Natural Hazards Risk Index),Expected population loss rate (Natural Hazards Risk Index),Percent individuals age 25 or over with less than high school degree in 2009,Percentage households below 100% of federal poverty line in 2009,Unemployment (percent) in 2009,Unemployment (percent) in 2010,Percent of individuals less than 100% Federal Poverty Line in 2010,Total population in 2009,Leaky underground storage tanks,DOT Travel Barriers Score,Share of properties at risk of flood in 30 years,Share of properties at risk of fire in 30 years,Share of the tract's land area that is covered by impervious surface or cropland as a percent,"Percent of individuals below 200% Federal Poverty Line, imputed and adjusted",Percent Black or African American,Percent American Indian / Alaska Native,Percent Asian,Percent Native Hawaiian or Pacific,Percent two or more races,Percent White,Percent Hispanic or Latino,Percent other races,Percent age under 10,Percent age 10 to 64,Percent age over 64,Percent of the Census tract that is within Tribal areas,Number of Tribal areas within Census tract for Alaska,Number of Tribal areas within Census tract,"Percent of the Census tract that is within Tribal areas, for display",Median household income as a percent of area median income,Life expectancy (years),Median household income as a percent of territory median income in 2009,Is there at least one abandoned mine in this census tract?,Income data has been estimated based on neighbor income,Is there at least one Formerly Used Defense Site (FUDS) in the tract?,Tract-level redlining score meets or exceeds 3.25,Identified as disadvantaged due to tribal overlap,Housing burden (percent) (percentile),Share of homes with no kitchen or indoor plumbing (percent) (percentile),Total population (percentile),Median household income (% of state median household income) (percentile),Current asthma among adults aged greater than or equal to 18 years (percentile),Coronary heart disease among adults aged greater than or equal to 18 years (percentile),Cancer (excluding skin ca

In [16]:
usa.shape

(74134, 261)

In [21]:
v1.head(10)

,csv_label,excel_label,Description,csv_field_type,calculation_notes,threshold_category,notes,geometry
0,Census tract 2010 ID,Census tract 2010 ID,GEOID10_TRACT,string,,,,None
1,County Name,County Name,County Name,string,,,,None
2,State/Territory,State/Territory,State/Territory,string,,,,None
3,Percent Black or African American alone,Percent Black or African American alone,Percent Black or African American,float,,,,None
4,Percent American Indian / Alaska Native,Percent American Indian / Alaska Native,Percent American Indian / Alaska Native,float,,,,None
5,Percent Asian,Percent Asian,Percent Asian,float,,,,None
6,Percent Native Hawaiian or Pacific,Percent Native Hawaiian or Pacific,Percent Native Hawaiian or Pacific,float,,,,None
7,Percent two or more races,Percent two or more races,Percent two or more races,float,,,,None
8,Percent White,Percent White,Percent White,float,,,,None
9,Percent Hispanic or Latino,Percent Hispanic or Latino,Percent Hispanic or Latino,float,,,,None


In [20]:
v1.shape

(156, 8)

In [22]:
v2.head()

,csv_label,excel_label,shapefile_label,Description,csv_field_type,calculation_notes,threshold_category,notes,geometry
0,Census tract 2010 ID,Census tract 2010 ID,,GEOID10_TRACT,string,,,,None
1,County Name,County Name,CF,County Name,string,,,,None
2,State/Territory,State/Territory,SF,State/Territory,string,,,,None
3,Percent Black or African American alone,Percent Black or African American alone,DM_B,Percent Black or African American,float,,,,None
4,Percent American Indian / Alaska Native,Percent American Indian / Alaska Native,DM_AI,Percent American Indian / Alaska Native,float,,,,None


In [23]:
v2.shape

(217, 9)